<a href="https://colab.research.google.com/github/hsgagne/Stock-Prediction-Machine-Learning-Project/blob/main/MF850.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
# don't need to do this every time, check if the data file is there
from google.colab import files
data_to_load = files.upload()

Saving finalproject_training.csv to finalproject_training.csv


In [ ]:
# import the training data
df = pd.read_csv("finalproject_training.csv")
df.head()

FileNotFoundError: ignored

In [ ]:
##############
# THINGS TO DO:
##############
# 1) How to handle missing data
# We could impute, exclude rows/columns, etc. ANSWER SimpleImputer
# No more than 10% of companies excluded
# 1.5) Also need to clean the data - what to do about date columns, organize the companies, etc.

# 2) How to split this data into training/validation/test
# ANSWER resampling lecture slide 19 - TimeSeriesSplit

# 3) how to do shrinkage
# - should we add a column of 1s for the intercept?
# - ridge vs. lasso vs. elastic net vs. other alternatives mentioned on shrinkage lecture slide 18
# ...the paper seems to prefer elastic net
# - should we standardize the data first? ANSWER: YES StandardScaler
##############
##############
#ToDo: Data & Feature Prep
#Here are some columns we might want to add to the features
# -some of his 
# 1) Valuation Ratio Columns
  #P/E, P/B, PEG(price to earnings growth)
# 2) Accounting Ratios
  #Debt/Assets
# 3) Technicals
  #Momentum
  #TTL shares repurchased (cshopq) -> % of mkt cap repurchased or something like that
# The paper mentions that the 3 most important features in their data was:
# price trends, liquidity, and volatility (top of p.2229)

#IMPUTION: Notes from (https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e)
#mean-median impution should only be used on continuous numerical values(need to think about this)
#K-NearestNeighbors & Random Forest: robust to missing values
# -RandomForest is good w non-linear data
#Predicion of Missing Values:
# 1) Split null values into Test Set (both for X and Y)
# 2) Only non-null values in Training Set 





In [ ]:
#re-indexing on companyid (non-unique index will have performance cost)
#https://stackoverflow.com/questions/16626058/what-is-the-performance-impact-of-non-unique-indexes-in-pandas
#df.index = df['comp_id']
#df = df.drop('comp_id',1)
#comp_id is a non-unique index() each index should only appear once in df
#can either leave index as 1,2,3,... or hash the entire row (not rly necessary)
df.index = df.apply(lambda x: hash(tuple(x)), axis = 1)

In [ ]:
#removing columns with dates or strings...and the monthly return which will be our target
features = list(df.columns)
str_features = ['date','sent_date','last_report','curcdq', 'datacqtr', 'datafqtr', 'costat']
#features.remove('m_ret') - don't do this anymore!
for i in str_features:
    features.remove(i)

In [ ]:
#implementing features from above, keeping in separate df for now
df["PtoE_f12"] = df['close']/df['epsf12']
df["PtoE_fxq"] = df['close']/df['epsfxq']
df["PtoE_pxq"] =df['close']/df['epspxq']
df["PtoE_x12"] =df['close']/df['epsx12']
df["PtoB"] = df["close"]/df["atq"]
df['lt_debt_r'] = df['dd1q']/df['dlttq'] #% of long term debt due in 1 year

#need to delete features used in compound ones, otherwise multicollinearity
feat_to_replace = ['epsf12','epsfxq','epspxq','atq','dd1q','dlttq']

df = df.replace([np.inf, -np.inf], np.nan)
df = df.sort_values(by = ['comp_id','date'])
for i in feat_to_replace:
  if i in features:
    features.remove(i)

dates = pd.Series(df['date'].values).unique()
companies = pd.Series(df['comp_id'].values).unique()
num_comp = len(companies)
df['mom1m']=0

mom = [np.log(df['close'].iloc[i] / df['close'].iloc[i-1]) if df['comp_id'].iloc[i] == df['comp_id'].iloc[i-1]\
       else 0 for i in range(len(df))]

#df['targ_ret'] = df['m_ret']

next_ret = [df['m_ret'].iloc[i] if df['comp_id'].iloc[i] == df['comp_id'].iloc[i-1]\
        else 0 for i in range(1,len(df))]
next_ret +=[0]


df['mom1m'] = mom
df['class'] = df['mom1m'] >= 0
df['target_ret'] = next_ret

new_features = ["PtoE_f12","PtoE_fxq","PtoE_pxq","PtoE_x12","lt_debt_r","PtoB",'mom1m']
features = features + new_features



In [ ]:
#re-sorting by date for time series splitting later
df = df.sort_values(by = ['date'])
df.tail(10)

,date,comp_id,close,m_volume,m_high,m_low,close_adj,m_volume_adj,m_high_adj,m_low_adj,m_divs,m_ret,sent_date,Bullish,Neutral,Bearish,Bullish8WeekMovAvg,SP500WeeklyHigh,SP500WeeklyLow,SP500WeeklyClose,last_report,fyearq,fqtr,curcdq,datacqtr,datafqtr,actq,atq,ceqq,cheq,chq,ciq,csh12q,cshfd12,cshfdq,cshopq,cshprq,dd1q,dlttq,dpq,...,nopiq,oiadpq,oibdpq,piq,rdipq,rectq,req,revtq,tfvaq,tfvceq,tfvlq,txdbclq,txtq,xaccq,xintq,xoprq,xrdq,fincfy,intpny,ivncfy,oancfy,txpdy,costat,dvpspq,dvpsxq,mkvaltq,ggroup,gind,gsector,gsubind,sic,PtoE_f12,PtoE_fxq,PtoE_pxq,PtoE_x12,PtoB,lt_debt_r,mom1m,class,target_ret
105048,2019-08-30,834140617009,211.40,134103755,244.51,211.000,42.280000,670518775.0,48.902000,42.200000,0.00,-0.058562,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,2,USD,2019Q2,2019Q2,10181.952,31872.597,5715.393,5082.746,4954.740,-359.848,173.1453,176.7210,176.654,NaN,176.654,2011.177,12309.747,578.572,...,-23.294,-50.113,528.459,-369.831,-47.026,1147.100,-6337.244,6349.676,1723.321,NaN,26.839,NaN,19.431,NaN,179.079,5821.217,276.872,1489.822,NaN,-547.388,224.000,NaN,A,0.00,0.00,40025.7083,2510,251020,25,25102010,3711,-54.625323,-91.515152,-91.515152,-56.074271,0.006633,0.163381,-0.060346,False,0.0
104033,2019-08-30,280861321973,1.20,28346247,1.58,1.125,1.200000,28346247.0,1.580000,1.125000,0.00,-0.111111,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,2,USD,2019Q2,2019Q2,122.459,233.143,-64.886,105.977,11.106,-22.965,106.0323,106.0323,106.272,NaN,106.272,1.668,245.278,0.910,...,0.691,-18.545,-17.635,-23.018,0.000,5.580,-1516.306,9.682,105.977,NaN,0.000,0.0,0.000,8.530,5.164,27.317,12.637,-1.584,9.610,-15.176,-52.520,NaN,A,0.00,0.00,668.4509,3520,352010,35,35201010,2835,-1.428571,-5.454545,-5.454545,-1.428571,0.005147,0.006800,-0.117783,False,0.0
104415,2019-08-30,833963543947,21.95,5770705,26.82,21.560,21.665174,5770705.0,26.471980,21.280234,0.00,-0.126889,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,2,USD,2019Q2,2019Q2,114.114,155.072,55.787,58.661,58.661,-0.978,16.0800,16.0800,16.290,0.039,16.290,1.206,61.408,2.109,...,-0.374,-1.984,0.125,-5.071,0.000,25.886,-2.543,44.247,0.000,NaN,0.000,0.0,-3.962,7.957,1.244,44.122,2.725,64.913,0.153,-3.317,-6.541,0.125,A,0.00,0.00,459.5754,4520,452030,45,45203010,3578,-13.222892,-313.571429,-313.571429,-13.222892,0.141547,0.019639,-0.135693,False,0.0
102846,2019-08-30,280952344072,22.91,13276365,25.94,20.340,22.910000,13276365.0,25.940000,20.340000,0.00,0.096697,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,2,USD,2019Q2,2019Q2,925.078,1811.287,919.619,160.546,160.546,35.699,67.0103,67.5045,67.077,0.041,66.578,29.479,445.488,17.632,...,3.408,54.283,71.915,46.189,0.000,373.469,266.394,946.375,NaN,NaN,NaN,0.0,10.490,NaN,5.574,874.460,0.000,-25.832,NaN,-93.657,129.312,NaN,A,0.00,0.00,1412.6832,2010,201070,20,20107010,5211,12.870787,43.226415,42.425926,12.798883,0.012648,0.066172,0.092303,True,0.0
104216,2019-08-30,833933254375,64.99,138332,71.90,64.900,63.225213,138332.0,69.947573,63.137657,0.35,-0.105067,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,1,USD,2019Q2,2019Q1,90.398,105.001,-65.148,73.903,73.903,5.369,4.1920,4.2150,4.206,0.000,4.206,1.065,152.930,0.310,...,0.387,9.448,9.758,7.185,0.000,14.974,-48.989,30.518,NaN,0.0,NaN,0.0,1.816,4.319,2.650,20.760,0.000,-0.401,4.969,-0.106,-1.036,0.060,A,0.35,0.35,330.2132,2530,253010,25,25301040,2013,12.426386,50.773437,50.773437,12.332068,0.618946,0.006964,-0.111007,False,0.0
105129,2019-08-30,1068785240541,34.42,6180409,48.63,33.540,34.420000,6180409.0,48.630000,33.540000,0.00,-0.224425,2019-08-29,0.261307,0.316583,0.422111,0.296648,2939.08,2834.97,2887.94,2019-06-30,2019,2,USD,2019Q2,2019Q2,1735.000,8223.000,4162.000,550.000,528.000,32.000,86.5000,87.7500,88.000,0.000,87.000,120.000,2454.000,176.000,...,39.000,36.000,212.000,46.000,0.000,959.000,2509.0

In [ ]:
df.iloc[0]

date          2016-08-31
comp_id         22439132
close              18.75
m_volume          207280
m_high             20.45
                 ...    
PtoB            0.515039
lt_debt_r            NaN
mom1m                  0
class               True
target_ret         0.032
Name: 2442, Length: 98, dtype: object

In [ ]:
###CODE FROM THE FEATURE SELECTOR PACKAGE####
#Output a list of 25 features to delete, based on 3 metrics
#1)High Collinearity
#2)Zero Importance Features
#3)Low Importance Features

fs_feat = ['cshfd12','m_high','gsector','gsubind','cshfdq','lseq','SP500WeeklyClose','gind','cshprq','ibmiiq','esopctq','m_low_adj','epsx12','tfvceq','ibq','txdbclq','rdipq','xoprq','ibcomq','PtoE_pxq','m_low','epspxq','fyearq','niq','SP500WeeklyLow']
for f in fs_feat:
  if f in features:
    features.remove(f)

In [ ]:
print(df[['date','comp_id','m_ret','target_ret']].head(50))

              date    comp_id     m_ret  target_ret
2442    2016-08-31   22439132  0.043406    0.032000
5289    2016-09-30   22439132  0.032000   -0.317829
8136    2016-10-31   22439132 -0.317829    0.072727
10983   2016-11-30   22439132  0.072727    0.027542
13830   2016-12-30   22439132  0.027542    0.012790
16677   2017-01-31   22439132  0.012790   -0.018736
19524   2017-02-28   22439132 -0.018736   -0.053942
22371   2017-03-31   22439132 -0.053942    0.023392
25218   2017-04-28   22439132  0.023392   -0.078571
28065   2017-05-31   22439132 -0.078571    0.057651
30912   2017-06-30   22439132  0.057651   -0.046446
33759   2017-07-31   22439132 -0.046446   -0.152344
36606   2017-08-31   22439132 -0.152344    0.000181
39453   2017-09-29   22439132  0.000181    0.106981
42300   2017-10-31   22439132  0.106981    0.027764
45147   2017-11-30   22439132  0.027764   -0.013515
47994   2017-12-29   22439132 -0.013515   -0.056497
50841   2018-01-31   22439132 -0.056497   -0.052740
53688   2018

In [ ]:
#save comp_id list
comp_id_list = df['comp_id']

#drop comp_id
features.remove("comp_id")

#ignore the first month - just testing right now
#there are two companies missing from the first month: 768323994503 and 70932507116
#the remaining months are evenly spaced for time series split
#I'm sure there's a better way to handle this
#or maybe not handle it at all because the scores aren't any better
#df = df.iloc[2845:]

#standardize the data
X = df.loc[:,features].values
#testing without this for now
X = StandardScaler().fit_transform(X)

#monthly returns untouched
Y = df['target_ret'].values

In [ ]:
#PCA using simple imputer
imp_1 = SimpleImputer(missing_values=np.nan, strategy='mean')
X_1 = imp_1.fit_transform(X)

In [ ]:
pca_2 = PCA(n_components=0.995,svd_solver='full')
pca_2.fit(X_1,Y)
print(len(pca_2.components_))

49


In [ ]:
#pca_1 = PCA()
#pca_1.fit(X_1,Y)
#print(np.cumsum(np.round(pca_1.explained_variance_ratio_, decimals=3)*100))
#Need 53 features to explain 100% of the variance
pca_2 = PCA(n_components=53)
pca_2.fit(X_1,Y)
print(np.cumsum(np.round(pca_2.explained_variance_ratio_, decimals=3)*100))

most_important = [np.abs(pca_2.components_[i]).argmax() for i in range(53)]
most_important_names = [features[most_important[i]] for i in range(53)]
unique = np.unique(np.array(most_important_names))
print(unique, len(unique))

#So we PCA gives 53 features to explain 100% of the variance
#These features are given by 48 columns
#I think we should just use these 48 columns
#I'll label this X_2, so final input for the regression model is X_2

X_2 = X = df.loc[:,unique].values
X_2 = StandardScaler().fit_transform(X_2)
X_2 = imp_1.fit_transform(X_2)

[ 31.5  35.9  39.9  43.7  47.3  50.8  54.   57.2  60.2  62.9  65.6  67.7
  69.7  71.7  73.6  75.4  77.2  79.   80.7  82.2  83.6  84.9  86.2  87.4
  88.5  89.6  90.6  91.5  92.2  92.9  93.6  94.2  94.8  95.4  95.9  96.4
  96.8  97.2  97.6  97.9  98.2  98.5  98.7  98.9  99.1  99.3  99.4  99.5
  99.6  99.7  99.8  99.9 100. ]
['Bullish' 'Bullish8WeekMovAvg' 'PtoB' 'PtoE_fxq' 'PtoE_x12'
 'SP500WeeklyHigh' 'ciq' 'close' 'csh12q' 'cshopq' 'dvpspq' 'dvpsxq'
 'fincfy' 'fqtr' 'ggroup' 'ibadjq' 'intanq' 'intpny' 'lctq' 'lltq' 'loq'
 'lt_debt_r' 'ltq' 'm_divs' 'm_high_adj' 'm_volume' 'mom1m' 'oancfy'
 'oiadpq' 'req' 'revtq' 'sic' 'tfvaq' 'tfvlq' 'txpdy' 'txtq' 'xaccq'
 'xintq' 'xrdq'] 39


In [ ]:
from sklearn.ensemble import RandomForestRegressor

def train(splits):
    tscv = TimeSeriesSplit(n_splits=splits)
    Y_prediction = {}
    i = 0
    for train_index, test_index in tscv.split(X_1):
        X_train, X_test = X_1[train_index], X_1[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
        random_forest = RandomForestRegressor(max_depth=4,n_jobs=-1,oob_score=True)
        random_forest.fit(X_train, Y_train)

        Y_prediction[i] = random_forest.predict(X_test)
        print("i:",i+1)
        print("Training score:",random_forest.score(X_train, Y_train))
        print("Test score:",random_forest.score(X_test, Y_test))
        i += 1
    return Y_prediction,Y_test

results,test_set = train(7) #need to fix this hard-coding

i: 1
Training score: 0.6857812756703234
Test score: -0.01413957581808356
i: 2
Training score: 0.6694071202656882
Test score: -0.6052767004421671
i: 3
Training score: 0.6335792883247586
Test score: -0.03573912982876659
i: 4
Training score: 0.5594122608913386
Test score: -0.14142309436848022
i: 5
Training score: 0.5522009040319493
Test score: 0.08041471981849735
i: 6
Training score: 0.537746220174629
Test score: -0.47874513052915346
i: 7
Training score: 0.4774752138704201
Test score: 0.03294671427101459


In [ ]:
#need to put these inside the function above if we want to use them
print(random_forest.oob_score_)
importances = pd.DataFrame({'feature':features,'importance':np.round(random_forest.feature_importances_,3)})
importances = importances.sort_values('importance',ascending=False).set_index('feature')
importances.head(15)

NameError: ignored

In [ ]:
#This code will go in the file that Max runs

import datetime
import dateutil.relativedelta
import csv

def write_csv(df,results,test_set):
  #get prediction date
  last_date = df.iloc[-1].date
  d = datetime.datetime.strptime(last_date, "%Y-%m-%d")
  prediction_date = d + dateutil.relativedelta.relativedelta(months=1)
  year_pred = prediction_date.year
  month_pred = prediction_date.month
  print(year_pred)
  print(month_pred)

  #arrange the data to be written to csv
  ret_rows = np.zeros((len(test_set),4))
  gof_rows = np.zeros((len(test_set),4))
  ret_rows[:,0] = year_pred
  gof_rows[:,0] = year_pred
  ret_rows[:,1] = month_pred
  gof_rows[:,1] = month_pred
  ret_rows[:,2] = comp_id_list[-13168:-1] #change this hard-coding later
  gof_rows[:,2] = comp_id_list[-13168:-1]
  ret_rows[:,3] = results[6] #change this hard-coding later
  gof_rows[:,3] = results[6] #change this to classication results later

  #write csvs
  ret_fields = ['year_pred','month_pred','comp_id','m_ret']
  gof_fields = ['year_pred','month_pred','comp_id','m_gof']
  ret_filename = 'returns.csv'
  gof_filename = 'grow-fall.csv'
  with open(ret_filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(ret_fields)
    csvwriter.writerows(ret_rows)
  with open(gof_filename, 'w') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(gof_fields)
    csvwriter.writerows(gof_rows)

write_csv(df,results,test_set)

2019
9


In [ ]:
#trying RandomizedSearchCV for parameter tuning
rf = RandomForestRegressor()
tscv = TimeSeriesSplit(n_splits=3)
for train_index, test_index in tscv.split(X_2):
  X_train, X_test = X_2[train_index], X_2[test_index]
  Y_train, Y_test = Y[train_index], Y[test_index]
  random_grid = {'max_depth': [3,4,5],
                  'n_estimators': [25,50,75]}
  rsc = RandomizedSearchCV(estimator = rf, param_distributions=random_grid,
                           cv=3,scoring='neg_mean_squared_error',verbose=2, n_jobs=-1)

  rsc_result = rsc.fit(X_train,Y_train)
  best_params = rsc_result.best_params_
  #rfr = RandomForestRegressor(max_depth = best_params['max_depth'], n_estimators = best_params['n_estimators'],n_jobs=-1)
  cv_results = rsc.cv_results_
  for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
#######PUTTING CLASSIFICATION FUNCTIONS BELOW########
#IMPORTS FOR ALL CLASSIFICATION FUNCTIONS
from sklearn import metrics
from sklearn.calibration import calibration_curve
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.model_selection import TimeSeriesSplit, train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

#import feature_selector

In [ ]:
##CODE TO RUN CLASSIFICATION FUNCTIONS###

##This Runs the Model Comparison and does the calibration plots##
#X_train,X_test,Y_train,Y_test = train_test_split(X_2,Y, test_size=.3)
#run_model_comparison(X_train,X_test,Y_train,Y_test)

##This Block will get a Random Forest Classifier w/ optimal hyperparams
Y_class = df['class']
depths = [14,15,16]
estimators = (94,95,96)
untrained_rf = rf_class_hyperparameters(X_2,Y,depths,estimators)
#Next we fit the untrained rf classifier
n_splits=5
rf = train_rf_class(X_2,Y,untrained_rf,n_splits)


In [ ]:
def rf_class_hyperparameters(X,Y,depths,estimators):
    """Uses GridSearchCV to estimate optimal hyperparameters for the Random Forest Classification
        Inputs: X,Y: Matrices of feature and class data
                Depths: an array of max_depths to test
                estimators: a tuple of estimators to test
    """
    #start with a default rf classifier
    rf = RandomForestClassifier()
    #split the training and test data, for fitting the grid result
    X_train,X_test,Y_train,Y_test = train_test_split(X_2,Y)

    #initialize gridsearch with given parameters
    gsc = GridSearchCV(estimator = rf,
                   param_grid ={
                           'max_depth':depths,
                           'n_estimators':estimators,
                           },
                           cv=3,verbose=0,
                           n_jobs=-1)
    
    #fit each combination of hyperparameters
    grid_result = gsc.fit(X_train,Y_train)

    #get the optimal parameters and print them out
    best_params = grid_result.best_params_
    print(best_params)

    #create and return a new random forest classifier
    rfr = RandomForestClassifier(max_depth = best_params['max_depth'], n_estimators = best_params['n_estimators'],n_jobs=-1)
    return rfr

def train_rf_class(X,Y,rfc,n_splits):
    """Uses timeseries split to train the random forest classifier
        Inputs: X, Y: matrices of feature and label data
                rfc: random forest classifier (use output of rf_class_hyperparameters)
                n_splits: number of time series splits to run
        Returns: a fitted random forest classifier
    """
    tscv = TimeSeriesSplit(n_splits=5)
    for train_index, test_index in tscv.split(X_2):
        X_train, X_test = X_2[train_index], X_2[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        rfc.fit(X_train,Y_train)
        print("Training score:",rfc.score(X_train, Y_train))
        print("Test score:",rfc.score(X_test, Y_test))
    
    return rfc

In [ ]:
###THIS WONT RUN HERE
def Feature_Selection(data,labels,missing_thresh,corr_thresh, cumulative_importance):
    """
    Feature_Selection Function:
        inputs:
            data: pd dataframe of feature data
            labels: np array of row labels
            missing_thresh,corr_thresh,cumulative importance: parameters for feature selector
    """
    #Create and Instance of Feature Selector
    fs = FeatureSelector(data=X,labels=Y)

    #identify all features to remove using feature selector class
    fs.identify_all(selection_params = {'missing_threshold': missing_thresh, 'correlation_threshold': corr_thresh, 
                                    'task': 'classification', 'eval_metric': 'auc', 
                                     'cumulative_importance': cumulative_importance})
    #create a new dataframe without the features flagged for removal
    train_removed = fs.remove(methods='all')
    #return the new df
    return train_removed


In [ ]:
def run_model_comparison(X_train,X_test,Y_train,Y_test):
    """
    Performs comparative analysis of classification models
    Plots calibration of multiple different classifiers, as well as an 
    accuracy, precision, and recall score
    
        Inputs:
            X_train,X_test,Y_train,Y_test: pre split matrices of test and training data
            
    """
    #setup the classifiers to test
    lr = LogisticRegression()
    gnb = GaussianNB()
    svc = LinearSVC(C=1.0)
    rfc = RandomForestClassifier(max_depth=15,n_estimators=95)
    knn = KNeighborsClassifier()
    #create a plot figure to store results in
    plt.figure(figsize=(10, 10))
    ax1 = plt.subplot2grid((3, 1), (0, 0), rowspan=2)
    #plot a 45 degree line that represents perfect calibration
    ax1.plot([0, 1], [0, 1], "k:", label="Perfectly calibrated")

    #for each classifier we created, perform the analysis
    for clf, name in [
                  (lr, 'Logistic Regression'),
                  (gnb, 'Naive Bayes'),
                  (svc,  'Support Vector Classification'),
                  (knn, 'K Nearest Neighbors'),
                  (rfc, 'Random Forest'),
                  ]:
        #perform the fit
        clf.fit(X_train, Y_train)

        #if the classifier has a predict_proba, use that as the prob of positives
        if hasattr(clf, "predict_proba"):
            prob_pos = clf.predict_proba(X_test)[:, 1]
        #otherwise we use the deficision function
        else:
            prob_pos = clf.decision_function(X_test)
            prob_pos = \
                (prob_pos - prob_pos.min()) / (prob_pos.max() - prob_pos.min())
        fraction_of_positives, mean_predicted_value = \
            calibration_curve(Y_test, prob_pos, n_bins=10)

        #plot the reliability curves
        ax1.plot(mean_predicted_value, fraction_of_positives, "s-",
             label="%s" % (name, ))
        #make a prediction on the test data for scoring
        Y_prediction = clf.predict(X_test)
        
        #print the current classifier name
        print(name)
        #print out the accuracy, precision, and recall
        print(clf.score(X_test,Y_test))
        print(metrics.precision_score(Y_test,Y_prediction))
        print(metrics.recall_score(Y_test,Y_prediction))
    
    #finish setting up the plot
    ax1.set_ylabel("Fraction of positives")
    ax1.set_ylim([-0.05, 1.05])
    ax1.legend(loc="lower right")
    ax1.set_title('Calibration plots  (reliability curve)')

    #show
    plt.tight_layout()
    plt.show()
    #returns nothing, outputs all results
    return

In [ ]:
###HARD CODED LIST FROM FEATURE SELECTION:
fs_features = ['cshopq', 'tfvaq', 'tfvceq', 'tfvlq', 'xaccq', 'xrdq', 'intpny', 'txpdy'] + ['m_low',
'm_high_adj',
'm_low_adj',
'SP500WeeklyLow',
'SP500WeeklyClose',
'cshfd12',
'cshfdq',
'cshprq',
'ibcomq',
'ibmiiq',
'ibq',
'lltq',
'ltq',
'niq',
'xoprq',
'gind',
'gsector',
'gsubind',
'PtoE_pxq'] + ['cshprq',
'fyearq',
'ibcomq',
'tfvceq',
'ibq',
'txdbclq',
'esopctq',
'rdipq',
'gsector']
print(fs_features)

['cshopq', 'tfvaq', 'tfvceq', 'tfvlq', 'xaccq', 'xrdq', 'intpny', 'txpdy', 'm_low', 'm_high_adj', 'm_low_adj', 'SP500WeeklyLow', 'SP500WeeklyClose', 'cshfd12', 'cshfdq', 'cshprq', 'ibcomq', 'ibmiiq', 'ibq', 'lltq', 'ltq', 'niq', 'xoprq', 'gind', 'gsector', 'gsubind', 'PtoE_pxq', 'cshprq', 'fyearq', 'ibcomq', 'tfvceq', 'ibq', 'txdbclq', 'esopctq', 'rdipq', 'gsector']


In [ ]:
#discarded code for Stochastic Gradient Descent, Decision Trees, K Nearest Neighbours
#SGD and Decision Trees give worse results, K Nearest Neighbours give similar to random forest 
#but takes ages 


from sklearn.ensemble import GradientBoostingRegressor

def train1(splits):
    tscv = TimeSeriesSplit(n_splits=splits)
    Y_prediction = {}
    i = 0
    for train_index, test_index in tscv.split(X_1):
        X_train, X_test = X_1[train_index], X_1[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
        gbr = GradientBoostingRegressor()
        gbr.fit(X_train, Y_train)

        Y_prediction[i] = gbr.predict(X_test)
        print("i:",i+1)
        print("Training score:",gbr.score(X_train, Y_train))
        print("Test score:",gbr.score(X_test, Y_test))
        i += 1
    return Y_prediction,Y_test

results,test_set = train1(7)


from sklearn.linear_model import SGDRegressor

def train2(splits):
    tscv = TimeSeriesSplit(n_splits=splits)
    Y_prediction = {}
    i = 0
    for train_index, test_index in tscv.split(X_1):
        X_train, X_test = X_1[train_index], X_1[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
        sgd = SGDRegressor()
        sgd.fit(X_train, Y_train)

        Y_prediction[i] = sgd.predict(X_test)
        print("i:",i+1)
        print("Training score:",sgd.score(X_train, Y_train))
        print("Test score:",sgd.score(X_test, Y_test))
        i += 1
    return Y_prediction,Y_test

results,test_set = train2(7)

from sklearn import tree

def train3(splits):
    tscv = TimeSeriesSplit(n_splits=splits)
    Y_prediction = {}
    i = 0
    for train_index, test_index in tscv.split(X_1):
        X_train, X_test = X_1[train_index], X_1[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
        clf = tree.DecisionTreeRegressor()
        clf.fit(X_train, Y_train)

        Y_prediction[i] = clf.predict(X_test)
        print("i:",i+1)
        print("Training score:",clf.score(X_train, Y_train))
        print("Test score:",clf.score(X_test, Y_test))
        i += 1
    return Y_prediction,Y_test

results,test_set = train3(7)

from sklearn.neighbors import KNeighborsRegressor

def train4(splits):
    tscv = TimeSeriesSplit(n_splits=splits)
    Y_prediction = {}
    i = 0
    for train_index, test_index in tscv.split(X_1):
        X_train, X_test = X_1[train_index], X_1[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
    
        knn = KNeighborsRegressor()
        knn.fit(X_train, Y_train)

        Y_prediction[i] = knn.predict(X_test)
        print("i:",i+1)
        print("Training score:",knn.score(X_train, Y_train))
        print("Test score:",knn.score(X_test, Y_test))
        i += 1
    return Y_prediction,Y_test

results,test_set = train4(7)